
Objective: Segment Equity bank customers into clusters and get profiles of each cluster.

Data:

1. Transaction
2. Demographics
3. End of day Balance
4. Loans

Steps:

1. Feature Extraction
2. Data cleaning
3. EDA
4. Preprocessing
5. Clustering using k-means
6. EDA to determine profiles per cluster

In [4]:
import re
import time
import sys
import os
import time
import calendar

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from itertools import islice
from configparser import ConfigParser
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [5]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import decomposition
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Feature Extraction

In [7]:
end_date = date(2021,12,31)

In [10]:
date_range = pd.date_range(end=end_date, periods=12, freq='M')
counter = 0
eod_list = []
for d in date_range:
    i = d.strftime('%Y%m')
    print(i)
    df  = pd.read_csv(f"C:/Users/kachase/Downloads/Clustering/Clustering/eod_data.csv", low_memory=False, 
                     converters = {'CIF_ID': lambda x:str(x)})
    eod_list.append(df)
    
eod = pd.concat(eod_list, ignore_index=True)

202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111


In [62]:
eod_agg['CIF_ID'] = eod_agg['CIF_ID'].astype(str)

In [16]:
df.head()

,CIF_ID,AVERAGE_MONTHLY_BALANCE,MAXIMUM_MONTHLY_BALANCE,MINIMUM_MONTHLY_BALANCE
0,54100000028,1.040000,1.04,1.04
1,54200000006,88517.198387,362933.65,3193.65
2,54200000132,10934.564516,31374.00,6.00
3,54200000408,1006.119234,14067.51,0.00
4,54200000862,364285.400968,387254.03,343964.03


In [63]:
eod_agg = eod.groupby(['CIF_ID']).agg({'AVERAGE_MONTHLY_BALANCE':['mean','std']})
eod_agg.columns = ['MONTHLY_AVG_BALANCE', 'STD_BALANCE']
eod_agg.reset_index(inplace=True)
eod_agg['CIF_ID'] = eod_agg['CIF_ID'].astype(str)

eod_agg.head()

,CIF_ID,MONTHLY_AVG_BALANCE,STD_BALANCE
0,54100000028,1.040000,0.0
1,54200000006,88517.198387,0.0
2,54200000132,10934.564516,0.0
3,54200000408,1006.119234,0.0
4,54200000862,364285.400968,0.0


In [ ]:
# eod_agg.to_csv("E:/data2/EGF/EOD/agg_eod.csv", index=False)

# CLEANING

In [18]:
len(eod_agg)

199202

In [19]:
eod_agg.CIF_ID.nunique()

199202

In [21]:
eod_agg.isnull().sum()

CIF_ID                 0
MONTHLY_AVG_BALANCE    0
STD_BALANCE            0
dtype: int64

In [24]:
# fill numerical null values with 0
numerical_features = [ 'MONTHLY_AVG_BALANCE', 'STD_BALANCE']

eod_agg[numerical_features] = eod_agg[numerical_features].fillna(0)

# cleaning and feature extraction of transactional data

In [27]:
df = pd.read_csv("C:/Users/kachase/Downloads/Clustering/Clustering/transactiona_data.csv")

In [28]:
df.head()

,Unnamed: 0,CIF_NUM,NUMBER_OF_TRANSACTIONAL_ACCOUNTS,COUNT_OF_CREDITS,COUNT_OF_DEBITS,MAX_CREDIT_VALUE,MAX_DEBIT_VALUE,AVERAGE_MONTHLY_CREDIT,AVERAGE_MONTHLY_CREDIT_COUNT,AVERAGE_MONTHLY_DEBIT,AVERAGE_MONTHLY_DEBIT.1,MAX_DAY_CREDIT,MAX_DAY_DEBIT
0,0,5.420000e+10,1,14.0,25.0,343000.0,220000.0,332050.00,1.750,355150.00,355150.00,2021-08-18,2021-08-28
1,1,5.420000e+10,1,8.0,10.0,5613.0,5190.0,5334.75,1.000,1406.25,1406.25,2021-08-24,2021-08-24
2,2,5.420000e+10,1,8.0,57.0,17952.0,10000.0,17063.00,1.000,15873.75,15873.75,2021-08-24,2021-08-30
3,3,5.420000e+10,1,8.0,8.0,5820.0,120.0,5531.25,1.000,120.00,120.00,2021-08-24,2021-08-24
4,4,5.420000e+10,2,3.0,3.0,14000.0,30000.0,4000.00,0.375,6500.00,6500.00,2021-08-18,2021-07-19


In [64]:
df.rename(columns = {'CIF_NUM':'CIF_ID'}, inplace = True)

In [65]:
df['CIF_ID'] = df['CIF_ID'].astype(str)

In [67]:
df.head()

,CIF_ID,NUMBER_OF_TRANSACTIONAL_ACCOUNTS,COUNT_OF_CREDITS,COUNT_OF_DEBITS,MAX_CREDIT_VALUE,MAX_DEBIT_VALUE,AVERAGE_MONTHLY_CREDIT,AVERAGE_MONTHLY_CREDIT_COUNT,AVERAGE_MONTHLY_DEBIT,AVERAGE_MONTHLY_DEBIT.1,MAX_DAY_CREDIT,MAX_DAY_DEBIT
0,54200000006.0,1,14.0,25.0,343000.0,220000.0,332050.00,1.750,355150.00,355150.00,2021-08-18,2021-08-28
1,54200000132.0,1,8.0,10.0,5613.0,5190.0,5334.75,1.000,1406.25,1406.25,2021-08-24,2021-08-24
2,54200000408.0,1,8.0,57.0,17952.0,10000.0,17063.00,1.000,15873.75,15873.75,2021-08-24,2021-08-30
3,54200000862.0,1,8.0,8.0,5820.0,120.0,5531.25,1.000,120.00,120.00,2021-08-24,2021-08-24
4,54200000876.0,2,3.0,3.0,14000.0,30000.0,4000.00,0.375,6500.00,6500.00,2021-08-18,2021-07-19


In [69]:
#df = df.drop("Unnamed: 0",axis=1)
#df.info()

In [87]:
df.isnull().sum()

CIF_ID                              0
NUMBER_OF_TRANSACTIONAL_ACCOUNTS    0
COUNT_OF_CREDITS                    0
COUNT_OF_DEBITS                     0
MAX_CREDIT_VALUE                    0
MAX_DEBIT_VALUE                     0
AVERAGE_MONTHLY_CREDIT              0
AVERAGE_MONTHLY_CREDIT_COUNT        0
AVERAGE_MONTHLY_DEBIT               0
AVERAGE_MONTHLY_DEBIT.1             0
MAX_DAY_CREDIT                      0
MAX_DAY_DEBIT                       0
dtype: int64

In [90]:
# fill numerical null values with 0
numerical_features = [ 'NUMBER_OF_TRANSACTIONAL_ACCOUNTS','COUNT_OF_CREDITS','COUNT_OF_DEBITS','MAX_CREDIT_VALUE','MAX_DEBIT_VALUE',
                      'AVERAGE_MONTHLY_CREDIT','AVERAGE_MONTHLY_CREDIT_COUNT','AVERAGE_MONTHLY_DEBIT',
                      'AVERAGE_MONTHLY_DEBIT.1','MAX_DAY_CREDIT','MAX_DAY_DEBIT']
      

df[numerical_features] = df[numerical_features].fillna(0)

# Cleaning and feature extraction of demographic data

In [33]:
demo_summary = pd.read_csv("C:/Users/kachase/Downloads/Clustering/Clustering/demographics.csv")
demo_summary.head()

,CIF_ID,SOL_ID,SALUTATION,GENDER,OCCUPATION,SEGMENTATION_CLASS,REGION,CITY,SECTOR,AGE,NUMBER_OF_MONTHS_SINCE_OPEN,HAS_CARD
0,54200054734,002,MR,M,NaN,RETAIL,NAI,223,TRADE,56,192.0,Yes
1,54200109824,017,MR,M,BNESS,RETAIL,NAI,223,CONS,40,192.0,Yes
2,54200120145,009,MR,M,OTH,RETAIL,CEN,283,F,51,192.0,Yes
3,54200134805,009,MR,M,NaN,RETAIL,CEN,283,NaN,63,192.0,Yes
4,54200139749,009,MR,M,OTH,RETAIL,CEN,495,DAIMA,56,192.0,No


In [75]:

demo['CIF_ID'] = demo['CIF_ID'].astype(str)

In [35]:
demo_summary.CIF_ID.nunique()

200000

In [45]:
demo = demo_summary[demo_summary['NUMBER_OF_MONTHS_SINCE_OPEN']>0].reset_index(drop=True)
len(demo)

186426

In [46]:
demo.isnull().sum()

CIF_ID                             0
SOL_ID                             0
SALUTATION                         0
GENDER                          8902
OCCUPATION                     36576
SEGMENTATION_CLASS               194
REGION                         12083
CITY                              26
SECTOR                         88296
AGE                                0
NUMBER_OF_MONTHS_SINCE_OPEN        0
HAS_CARD                           0
dtype: int64

In [48]:
demo['REGION'].fillna('UNKNOWN', inplace=True)

In [49]:
# segmentation class
seg_class=['RETAIL', 'BUSINESS', 'AGRIC', 'MICRO', 'CONSM','CORPT','SME', 'INSTIT', 'PUBLI']
demo.loc[~demo['SEGMENTATION_CLASS'].isin(seg_class),'SEGMENTATION_CLASS'] = 'OTHERS'

In [50]:
demo['GENDER'] =  demo['GENDER'].str.upper()
demo.loc[~demo['GENDER'].isin(['F','M']), 'GENDER'] = 'UNKNOWN'

In [51]:
# Occupation
occ_class=['TECH','STUD','FARM', 'BNESS']
demo['OCCUPATION'].unique()
demo.loc[~demo['OCCUPATION'].isin(occ_class),'OCCUPATION']='OTHERS'

In [52]:
demo.drop(['SECTOR', 'CITY'], axis=1, inplace=True)

In [53]:
demo.isnull().sum()

CIF_ID                         0
SOL_ID                         0
SALUTATION                     0
GENDER                         0
OCCUPATION                     0
SEGMENTATION_CLASS             0
REGION                         0
AGE                            0
NUMBER_OF_MONTHS_SINCE_OPEN    0
HAS_CARD                       0
dtype: int64

In [91]:
# fill numerical null values with 0
numerical_features = ['NUMBER_OF_MONTHS_SINCE_OPEN', 'AGE']
      

demo[numerical_features] = demo[numerical_features].fillna(0)

In [72]:
eod_agg.dtypes

CIF_ID                  object
MONTHLY_AVG_BALANCE    float64
STD_BALANCE            float64
dtype: object

In [77]:
demo.dtypes

CIF_ID                          object
SOL_ID                          object
SALUTATION                      object
GENDER                          object
OCCUPATION                      object
SEGMENTATION_CLASS              object
REGION                          object
AGE                              int64
NUMBER_OF_MONTHS_SINCE_OPEN    float64
HAS_CARD                        object
dtype: object

In [74]:
df.dtypes

CIF_ID                               object
NUMBER_OF_TRANSACTIONAL_ACCOUNTS      int64
COUNT_OF_CREDITS                    float64
COUNT_OF_DEBITS                     float64
MAX_CREDIT_VALUE                    float64
MAX_DEBIT_VALUE                     float64
AVERAGE_MONTHLY_CREDIT              float64
AVERAGE_MONTHLY_CREDIT_COUNT        float64
AVERAGE_MONTHLY_DEBIT               float64
AVERAGE_MONTHLY_DEBIT.1             float64
MAX_DAY_CREDIT                       object
MAX_DAY_DEBIT                        object
dtype: object

In [80]:
data  = pd.merge(data,demo, on='CIF_ID', how='left')
data  = pd.merge(data, eod_agg, on='CIF_ID', how='left')
data  = pd.merge(data, df, on='CIF_ID', how='left')

In [81]:
data.head()

,CIF_ID,NUMBER_OF_TRANSACTIONAL_ACCOUNTS_x,COUNT_OF_CREDITS_x,COUNT_OF_DEBITS_x,MAX_CREDIT_VALUE_x,MAX_DEBIT_VALUE_x,AVERAGE_MONTHLY_CREDIT_x,AVERAGE_MONTHLY_CREDIT_COUNT_x,AVERAGE_MONTHLY_DEBIT_x,AVERAGE_MONTHLY_DEBIT.1_x,...,COUNT_OF_CREDITS,COUNT_OF_DEBITS,MAX_CREDIT_VALUE,MAX_DEBIT_VALUE,AVERAGE_MONTHLY_CREDIT,AVERAGE_MONTHLY_CREDIT_COUNT,AVERAGE_MONTHLY_DEBIT,AVERAGE_MONTHLY_DEBIT.1,MAX_DAY_CREDIT,MAX_DAY_DEBIT
0,54200000006.0,1,14.0,25.0,343000.0,220000.0,332050.00,1.750,355150.00,355150.00,...,14.0,25.0,343000.0,220000.0,332050.00,1.750,355150.00,355150.00,2021-08-18,2021-08-28
1,54200000132.0,1,8.0,10.0,5613.0,5190.0,5334.75,1.000,1406.25,1406.25,...,8.0,10.0,5613.0,5190.0,5334.75,1.000,1406.25,1406.25,2021-08-24,2021-08-24
2,54200000408.0,1,8.0,57.0,17952.0,10000.0,17063.00,1.000,15873.75,15873.75,...,8.0,57.0,17952.0,10000.0,17063.00,1.000,15873.75,15873.75,2021-08-24,2021-08-30
3,54200000862.0,1,8.0,8.0,5820.0,120.0,5531.25,1.000,120.00,120.00,...,8.0,8.0,5820.0,120.0,5531.25,1.000,120.00,120.00,2021-08-24,2021-08-24
4,54200000876.0,2,3.0,3.0,14000.0,30000.0,4000.00,0.375,6500.00,6500.00,...,3.0,3.0,14000.0,30000.0,4000.00,0.375,6500.00,6500.00,2021-08-18,2021-07-19


# standard features

In [94]:
#Remove constant features:This is because features with single unique values do not help the model generalize well as they have variance/standard deviation of 0
def find_constant_features(dataFrame):
    constant = []
    for column in list(dataFrame.columns):
        if dataFrame[column].unique().size < 2:
            constant.append(column)
    return constant
constant = find_constant_features(data)
print(constant)
data.drop(constant, axis =1, inplace = True)
data.shape

[]


(153372, 34)

E.D.A